In [1]:
#include <vector>
#include <string>
#include <mutex>

#include <iostream>

class write_ahead_log{
friend class shared_state;
public:
    void write(unsigned& index, unsigned tag, const std::string& blob){
        std::unique_lock lock(mut);
        log.push_back(std::pair(tag,blob));
        index=log.size()-1; 
    }
    bool read(unsigned index, unsigned& tag, std::string& blob){
        std::unique_lock lock(mut);
        if(index<log.size()){tag=log[index].first; blob=log[index].second; return true;}
        return false;
    }
private:
    std::vector<std::pair<unsigned,std::string>> log;
    std::mutex mut;
};

In [2]:
#include <functional>
#include <iostream>

class tasks{
public:
    tasks(write_ahead_log&l):log(l){}
    void exec_goon( std::function<void(std::ostream&)>exec,
                    std::function<void(std::istream&)>goon){}
    void exec_goon_if( bool condition,
                    std::function<void(std::ostream&)>exec,
                    std::function<void(std::istream&)>goon){}

    void barrier(){}
private:
    write_ahead_log& log;
};

In [3]:
#include <iostream>
#include <thread>
#include <atomic>
#include <iostream>

{
    std::atomic_int PID = 0;
    int NUM_THREADS = 10;
    int ARRAY_SIZE = 10;
    
    write_ahead_log wal;
    std::vector<std::thread> threads(NUM_THREADS);
    
    for(auto& t:threads)t=std::thread([&]{ int pid = PID++;
    //////////////// inside a 'process' ////////////////
    int N  [ARRAY_SIZE];
    int NxN[ARRAY_SIZE];
                                           
    tasks tt(wal);
                                           
    for(int i=0; i<ARRAY_SIZE; i++){
        tt.exec_goon_if( pid == 0,
            [&](std::ostream&out){
                N[i] = i+1; out << N[i];
            },
            [&](std::istream&in){
                in >> N[i];
            }
        );
    }
    tt.barrier();

    for(int i=0; i<ARRAY_SIZE; i++){
        tt.exec_goon(
            [&](std::ostream&out){
                NxN[i] = N[i]*N[i]; out << N[i]xN[i];
            },
            [&](std::istream&in){
                in >> NxN[i];
            }
        );
    }
    tt.barrier();

    if(pid==0){// .. or any other pid < NUM_THREADS 
        for(int i=0; i<ARRAY_SIZE; i++)
            std::cout << i << " * " << i  << " = " 
                << NxN[i] << std::endl;
    }
    ////////////////////////////////////////////////////
    }); for(auto& t:threads) t.join();
    std::cout << "Success!" << std::endl;
}

0 * 0 = 0
1 * 1 = 0
2 * 2 = 0
3 * 3 = 0
4 * 4 = 0
5 * 5 = 0
6 * 6 = 0
7 * 7 = 0
8 * 8 = 4
9 * 9 = 0
Success!
